In [14]:
import json
import os
import numpy as np

def load_existing_results(file_path="forecasting_results.json"):
    """
    Load existing results from a JSON file.
    Returns an empty dictionary if the file doesn't exist.
    """
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return json.load(f)
    return {}


def save_results_to_json(data, file_path="forecasting_results.json"):
    """
    Save the results dictionary to a JSON file, handling NumPy data types.
    """

    # Handle NumPy data types (recursive conversion)
    def convert_numpy(obj):
        if isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(i) for i in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert arrays to lists
        else:
            return obj

    # Convert data and save to JSON
    data = convert_numpy(data)
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"✅ Results saved to {file_path}")



def store_results(dataset_name, horizons, horizon_value, experiment_type, backbone, mae_result, file_path="forecasting_results.json"):
    """
    Store MAE results for a given experiment type (stl_mae, mtl_mae, global_mae) per horizon.

    Args:
    - dataset_name (str): Name of the dataset (e.g., 'Solar', 'Air Quality').
    - horizons (list): List of horizon values (e.g., [1, 2, 4, 8, 16]).
    - horizon_value (int): The horizon corresponding to the mae_result provided.
    - experiment_type (str): One of ['stl_mae', 'mtl_mae', 'global_mae'].
    - backbone (str): Model backbone name (e.g., 'Deep_LSTM', 'simple_transformer').
    - mae_result (list): MAE values for the current horizon (list of floats).
    - file_path (str): JSON file to store the results.

    Returns:
    - None
    """
    # Load existing results
    results_dict = load_existing_results(file_path)

    # Create dataset entry if it doesn't exist
    dataset_key = f"{dataset_name}_{backbone}"
    if dataset_key not in results_dict:
        results_dict[dataset_key] = {
            "horizons": horizons,
            "mtl": [[] for _ in horizons],
            "global": [[] for _ in horizons],
            "independent": [[] for _ in horizons]
        }

    # Find index for the given horizon
    try:
        horizon_index = horizons.index(horizon_value)
    except ValueError:
        raise ValueError(f"⚠️ Horizon value {horizon_value} not found in {horizons}.")

    # Append the mae_result to the correct horizon
    results_dict[dataset_key][experiment_type][horizon_index].extend(mae_result)

    # Save updated results
    save_results_to_json(results_dict, file_path)

In [15]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import dateutil.parser
import matplotlib.pyplot as plt

def df_to_X_y(df, features, target, window_size=32, horizon=1):
    if target not in features:
        features = [target] + features

    data = df[features].to_numpy()
    target_data = df[target].to_numpy()

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i + window_size])
        y.append(target_data[i + window_size: i + window_size + horizon])

    return np.array(X), np.array(y)

In [16]:
# --------------------------- #
#          LSTM Model         #
# --------------------------- #

class LSTMModel(nn.Module):
    def __init__(self, num_features, time_window, output_window, num_labels, num_layers=2, hidden_size=16, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.output_window = output_window
        self.num_labels = num_labels

        self.lstm = nn.LSTM(num_features,
                            hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, num_labels * output_window)

    def forward(self, x):
        B, T, D = x.shape  
        x_, _ = self.lstm(x)
        last_hidden = x_[:, -1, :]  # Take last time step's hidden state
        x_ = self.fc(last_hidden)
        x_ = x_.reshape(B, self.output_window, self.num_labels)
        return x_

In [26]:

# --------------------------- #
#        Data Preprocessing   #
# --------------------------- #
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import dateutil.parser
class BalancedDataLoaderIterator:
    '''
    This class iterates over multiple PyTorch dataloaders in a balanced way, 
    ensuring that all dataloaders contribute to the training equally. 
    It uses random sampling to choose from multiple dataloaders while 
    handling different dataset lengths.
    '''
    def __init__(self, dataloaders):
        self.dataloaders = dataloaders

        self.num_dataloaders = len(dataloaders)

        max_length = max(len(dataloader) for dataloader in dataloaders)

        length_list = [len(dataloader) for dataloader in dataloaders]
        print("data loader length:", length_list)
        print("max dataloader length:", max_length,
              "epoch iteration:", max_length * self.num_dataloaders)
        self.total_length = max_length * self.num_dataloaders
        self.current_iteration = 0
        self.probabilities = torch.ones(
            self.num_dataloaders, dtype=torch.float) / self.num_dataloaders

    def __iter__(self):
        self.iterators = [iter(dataloader) for dataloader in self.dataloaders]
        self.current_iteration = 0
        return self

    def __next__(self):
        if self.current_iteration >= self.total_length:
            raise StopIteration

        chosen_index = torch.multinomial(self.probabilities, 1).item()
        try:
            sample = next(self.iterators[chosen_index])
        except StopIteration:
            self.iterators[chosen_index] = iter(self.dataloaders[chosen_index])
            sample = next(self.iterators[chosen_index])

        self.current_iteration += 1
        return sample, chosen_index

    def __len__(self):
        return self.total_length
    
    
def load_all_sites_balanced(
    base_dir='../processed_ds/air_quality_cluster/', 
    features= [],
    target = '',
    window_size=32, 
    horizon=16, 
    batch_size=16,
    min_date=None, 
    max_date=None,
    device="cpu"
):
    """
    Loads and preprocesses time series data for all sites, ensuring balanced training across sites.

    Args:
    - base_dir (str): Path to the directory containing site folders.
    - window_size (int): Number of past time steps for input.
    - horizon (int): Number of future steps to predict.
    - batch_size (int): Batch size for DataLoader.
    - min_date (str or datetime, optional): Start date for filtering data.
    - max_date (str or datetime, optional): End date for filtering data.

    Returns:
    - balanced_train_loader: Balanced DataLoader across all sites for training.
    - site_val_loaders: Dictionary of validation DataLoaders per site.
    - site_test_loaders: Dictionary of test DataLoaders per site.
    """
    
    # Get the list of site directories
    site_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    site_dirs.sort()

    train_loaders = []
    site_val_loaders = {}
    site_test_loaders = {}

    for site in site_dirs:
        site_path = os.path.join(base_dir, site)
        csv_files = [f for f in os.listdir(site_path) if f.endswith('.csv')]
        csv_files.sort()

        for csv_file in csv_files:
            file_path = os.path.join(site_path, csv_file)
            df_site = pd.read_csv(file_path)

            # Convert date column to datetime if it exists
            if 'date' in df_site.columns:
                df_site['date'] = pd.to_datetime(df_site['date'])

                # Filter based on min_date and max_date
                if min_date:
                    min_date = dateutil.parser.parse(min_date) if isinstance(min_date, str) else min_date
                    df_site = df_site[df_site['date'] >= min_date]

                if max_date:
                    max_date = dateutil.parser.parse(max_date) if isinstance(max_date, str) else max_date
                    df_site = df_site[df_site['date'] <= max_date]

                # Drop date column after filtering
                df_site.drop(columns=['date'], inplace=True)

            # Perform 80-20 train-test split
            train_size = int(0.8 * len(df_site))
            train_df = df_site.iloc[:train_size]
            test_df = df_site.iloc[train_size:]

            # Split train_df further into Train (80%) and Validation (20%)
            val_size = int(0.2 * len(train_df))  # 16% of full dataset
            train_df, val_df = train_df.iloc[:-val_size], train_df.iloc[-val_size:]

            print(f"Site: {site} | Train: {len(train_df)} | Validation: {len(val_df)} | Test: {len(test_df)}")

            # Standardize data per site (avoid data leakage)
            train_mean, train_std = train_df.mean(), train_df.std()
            train_df = (train_df - train_mean) / (train_std + 1e-8)
            val_df = (val_df - train_mean) / (train_std + 1e-8)
            test_df = (test_df - train_mean) / (train_std + 1e-8)

            # Convert DataFrame to NumPy arrays for LSTM
            X_train, y_train = df_to_X_y(train_df, features, target, window_size, horizon)
            X_val, y_val = df_to_X_y(val_df,features, target ,window_size, horizon)
            X_test, y_test = df_to_X_y(test_df, features, target ,window_size, horizon)

            # Convert to PyTorch tensors
            # train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
            # val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
            # test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
            
            train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device))
            val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32).to(device), torch.tensor(y_val, dtype=torch.float32).to(device))
            test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.float32).to(device))

            # Create DataLoaders
            train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
            val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, drop_last=True)
            test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)

            # Store loaders for this site
            train_loaders.append(train_loader)
            site_val_loaders[site] = val_loader
            site_test_loaders[site] = test_loader

    # Create Balanced DataLoader for Training
    balanced_train_loader = BalancedDataLoaderIterator(train_loaders)

    return balanced_train_loader, site_val_loaders, site_test_loaders

def train_model_balanced(model, balanced_train_loader, site_val_loaders, num_epochs=30, lr=1e-3, wd=1e-5):
    """
    Trains an LSTM model using a balanced data loader (across multiple sites) and evaluates on
    separate site-specific validation loaders to prevent data leakage.

    Args:
      model (nn.Module): The LSTM model.
      balanced_train_loader (BalancedDataLoaderIterator): Iterator that yields (batch, chosen_index)
            from a list of site-specific training DataLoaders.
      site_val_loaders (dict): A dictionary mapping site identifiers to validation DataLoaders.
      num_epochs (int): Number of training epochs.
      lr (float): Learning rate.
      wd (float): Weight decay.
      
    Returns:
      train_losses (list): Average training loss per epoch.
      val_losses (list): Average validation loss per epoch (aggregated across sites).
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    device = "cuda"
    model.to(device)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        
        # Iterate over the balanced training data
        for (batch, chosen_index) in balanced_train_loader:
            batch_x, batch_y = batch
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            output = model(batch_x)
            # Ensure that output and target shapes match: unsqueeze target if necessary
            loss = criterion(output, batch_y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            
            running_train_loss += loss.item()
        
        avg_train_loss = running_train_loss / len(balanced_train_loader)
        train_losses.append(avg_train_loss)
        
        # -------------------- Validation (no data leak) -------------------- #
        model.eval()
        running_val_loss = 0.0
        val_batches = 0
        
        with torch.no_grad():
            # Loop over each site's validation loader separately
            for site, val_loader in site_val_loaders.items():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    output = model(batch_x)
                    loss = criterion(output, batch_y.unsqueeze(-1))
                    running_val_loss += loss.item()
                    val_batches += 1
        
        avg_val_loss = running_val_loss / val_batches if val_batches > 0 else float('inf')
        val_losses.append(avg_val_loss)
        
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    
    print("Training complete!")
         # -------------------- Plot Loss Curves -------------------- #

    # plt.figure(figsize=(8, 5))
    # plt.plot(range(1, num_epochs + 1), train_losses, label="Training Loss", marker='o')
    # plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss", marker='s')
    
    # plt.xlabel("Epoch")
    # plt.ylabel("Loss")
    # plt.title("Training and Validation Loss Curves")
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    
    return train_losses, val_losses

def evaluate_model_balanced(model, site_test_loaders):
    """
    Evaluates the LSTM model on test data from each site separately to avoid data leakage.

    Args:
      model (nn.Module): Trained LSTM model.
      site_test_loaders (dict): Dictionary mapping site identifiers to test DataLoaders.

    Returns:
      site_mae_list (list): List containing MAE for each site.
    """
    model.eval()
    criterion = nn.L1Loss(reduction='mean')
    site_mae_list = []

    with torch.no_grad():
        for site, test_loader in site_test_loaders.items():
            site_preds, site_targets = [], []
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")
                predictions = model(batch_x)
                site_preds.append(predictions.cpu())
                site_targets.append(batch_y.unsqueeze(-1).cpu())
            
            # Concatenate predictions and targets for full-site evaluation
            site_preds = torch.cat(site_preds, dim=0)
            site_targets = torch.cat(site_targets, dim=0)
            site_mae = criterion(site_preds, site_targets).item()

            print(f"Site: {site}, Test MAE: {site_mae:.4f}")
            site_mae_list.append(site_mae)
    
    return site_mae_list


In [24]:

import os
import torch
import numpy as np

def run_global_model_experiment_lstm():
    """
    Runs a global LSTM model experiment by concatenating all site data for multiple datasets and horizons.
    Appends site-wise and mean MAE results to output.txt.
    """
    datasets = [
        {
            'name': 'Air Quality',
            'features': ['PM2.5', 'OT', 'PM10', 'NO2'],
            'target': 'PM2.5',
            'directory': "../processed_ds/air_quality_cluster/",
            'min_date': "2014-09-01",
            'max_date': "2014-11-12 19:00",
            'num_features': 4
        },
        {
            'name': 'Solar',
            'directory': "../processed_ds/solar/",
            'features': ['loc-1', 'loc-2', 'loc-3', 'loc-4'],
            'target': 'loc-1',
            'min_date': "2006-09-01",
            'max_date': "2006-09-08 4:50",
            'num_features': 4
        },
        {
            'name': 'Crypto',
            'directory': "../processed_ds/crypto-data/",
            'features': ['Open', 'High', 'Low', 'OT', 'Volume'],
            'target': 'OT',
            'min_date': "2018-04-01",
            'max_date': "2018-06-15",
            'num_features': 5
        },
        # {
        #     'name': 'Sales',
        #     'directory': "../processed_ds/stores_data/",
        #     'min_date': "2013-01-16",
        #     'max_date': "2015-07-31",
        #     'num_features': 7
        # }
    ]

    horizons = [1, 2, 4, 8, 16]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seq_len = 32
    batch_size = 32
    num_epochs = 30

    for dataset in datasets:
        print(f"\n==================== Dataset: {dataset['name']} ====================")

        for horizon in horizons:
            print(f"\n--- Running Global Model for Horizon: {horizon} ---")

            # Load balanced data (all sites concatenated)
            balanced_train_loader, val_loaders, test_loaders = load_all_sites_balanced(
                base_dir=dataset['directory'],
                features=dataset['features'],
                target=dataset['target'],
                window_size=seq_len,
                horizon=horizon,
                batch_size=batch_size,
                min_date=dataset['min_date'],
                max_date=dataset['max_date'],
                device=device
            )

            # Global LSTM model definition
            model = LSTMModel(
                num_features=dataset['num_features'],
                time_window=seq_len,
                output_window=horizon,
                num_labels=1,
                num_layers=2,
                hidden_size=8,
                dropout=0.5
            )
            total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print("Total trainable parameters:", total_params)
            model.to(device)

            # Train the global model
            train_model_balanced(
                model, 
                balanced_train_loader, 
                val_loaders, 
                num_epochs=num_epochs, 
                lr=1e-4, 
                wd=1e-5
            )

            # Evaluate the model on test data
            mae_per_site = evaluate_model_balanced(model, test_loaders)
            avg_mae = np.mean(mae_per_site)

            # Output results
            print(f"Site-wise MAE for {dataset['name']} at horizon {horizon}: {mae_per_site}")
            print(f"Average MAE for {dataset['name']} at horizon {horizon}: {avg_mae:.4f}")

            # Append results to output.txt
            with open("output_test.txt", "a") as f:
                f.write("\n==================== Simple GLOBAL LSTM MODEL RESULTS ====================\n")
                f.write(f"Dataset: {dataset['name']}\n")
                f.write(f"Horizon: {horizon}\n")
                f.write(f"MAE per site: {mae_per_site}\n")
                f.write(f"Mean MAE: {avg_mae:.4f}\n")
            store_results(
                dataset_name=dataset['name'],
                horizons=[1,2,4,8,16],
                experiment_type='global',
                mae_result=mae_per_site,
                backbone='simple_lstm',
                horizon_value=horizon
            )

    print("\n🏆 All Global LSTM Forecasting Model experiments are complete! 🏆")


In [27]:
run_global_model_experiment_lstm()


==================== Dataset: Air Quality ====================

--- Running Global Model for Horizon: 1 ---
Site: site-1 | Train: 1119 | Validation: 279 | Test: 350
Site: site-10 | Train: 1119 | Validation: 279 | Test: 350
Site: site-11 | Train: 1119 | Validation: 279 | Test: 350
Site: site-12 | Train: 1119 | Validation: 279 | Test: 350
Site: site-2 | Train: 1119 | Validation: 279 | Test: 350
Site: site-3 | Train: 1119 | Validation: 279 | Test: 350
Site: site-4 | Train: 1119 | Validation: 279 | Test: 350
Site: site-5 | Train: 1119 | Validation: 279 | Test: 350
Site: site-6 | Train: 1119 | Validation: 279 | Test: 350
Site: site-7 | Train: 1119 | Validation: 279 | Test: 350
Site: site-8 | Train: 1119 | Validation: 279 | Test: 350
Site: site-9 | Train: 1119 | Validation: 279 | Test: 350
data loader length: [33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33]
max dataloader length: 33 epoch iteration: 396
Total trainable parameters: 1033
Epoch 1 | Train Loss: 1.0049 | Val Loss: 2.5852
Epoch 2